DATA ENGINEERING - Qui TRình xử lí dữ liệu


## 1. Khởi tạo & Đọc dữ liệu (Data Ingestion)

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType

# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("KingCounty_ML") \
    .getOrCreate()

# Đọc dữ liệu
input_path = "kc_house_data.csv"
df = spark.read.csv(input_path, header=True, inferSchema=True)

print(f"📊 Dữ liệu gốc: {df.count():,} dòng.")
df.printSchema()

📊 Dữ liệu gốc: 21,613 dòng.
root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)



## 2. Làm sạch dữ liệu và xử lí outlier
Dựa trên kết quả Phân tích khám phá (EDA), áp dụng chiến lược làm sạch sau:

* **Loại cột id** vì không có ý nghĩa khi huấn luyện mô hình

* **Loại bỏ:** Các dòng dữ liệu lỗi nhập liệu (VD: 33 phòng ngủ), các dòng thiếu thông tin cơ bản (giá = 0, diện tích = 0).
* **Giữ lại:** Các giá trị `0` hợp lệ (như `view=0`, `sqft_basement=0`) và các căn biệt thự diện tích lớn (Outliers giá trị thực).

In [ ]:
df= df.drop("id")

In [ ]:
# Lọc bỏ các dòng dữ liệu không hợp lệ về mặt vật lý
df_clean = df.filter(
    (F.col("bedrooms") != 33) &       # Xóa lỗi nhập liệu (Typo: 33 phòng)
    (F.col("bedrooms") > 0) &         # Nhà phải có ít nhất 1 phòng ngủ
    (F.col("bathrooms") > 0) &        # Nhà phải có phòng tắm
    (F.col("sqft_living") > 0) &      # Diện tích phải > 0
    (F.col("price") > 0)              # Giá tiền phải > 0
)

# Thống kê lượng dữ liệu bị loại bỏ
raw_count = df.count()
clean_count = df_clean.count()
print(f"🧹 Dữ liệu sau khi làm sạch: {clean_count:,} dòng.")
print(f"❌ Đã loại bỏ: {raw_count - clean_count} dòng lỗi.")

🧹 Dữ liệu sau khi làm sạch: 21,596 dòng.
❌ Đã loại bỏ: 17 dòng lỗi.


## 3. Feature Engineering
* **Trích xuất thông tin:** Tách tháng bán (`sell_month`) từ cột `date` để mô hình học được tính mùa vụ.
* **Chuẩn hóa kiểu:** Ép kiểu `Integer` và `Double` chuẩn xác để tối ưu hóa bộ nhớ và tính toán.

In [ ]:
df_processed = df_clean.withColumn("date_parsed", F.to_date(F.substring(F.col("date"), 1, 8), "yyyyMMdd")) \
                       .withColumn("sell_month", F.month(F.col("date_parsed"))) \
                       .withColumn("grade", F.col("grade").cast(IntegerType())) \
                       .withColumn("condition", F.col("condition").cast(IntegerType())) \
                       .withColumn("view", F.col("view").cast(IntegerType())) \
                       .withColumn("waterfront", F.col("waterfront").cast(IntegerType())) \
                       .withColumn("bedrooms", F.col("bedrooms").cast(IntegerType())) \
                       .withColumn("price", F.col("price").cast(DoubleType())) \
                       .withColumn("sqft_living", F.col("sqft_living").cast(DoubleType())) \
                       .withColumn("lat", F.col("lat").cast(DoubleType())) \
                       .withColumn("long", F.col("long").cast(DoubleType()))

# Loại bỏ cột trung gian
df_final = df_processed.drop("date_parsed", "date")

print("✅ Đã tạo xong đặc trưng 'sell_month' và chuẩn hóa Schema.")
df_final.select("price", "grade", "sell_month", "lat", "long").show(5)

✅ Đã tạo xong đặc trưng 'sell_month' và chuẩn hóa Schema.
+--------+-----+----------+-------+--------+
|   price|grade|sell_month|    lat|    long|
+--------+-----+----------+-------+--------+
|221900.0|    7|        10|47.5112|-122.257|
|538000.0|    7|        12| 47.721|-122.319|
|180000.0|    6|         2|47.7379|-122.233|
|604000.0|    7|        12|47.5208|-122.393|
|510000.0|    8|         2|47.6168|-122.045|
+--------+-----+----------+-------+--------+
only showing top 5 rows


In [ ]:
print(f"Dữ liệu hiện có {df_processed.count():,} dòng và {len(df_processed.columns)} cột.")

Dữ liệu hiện có 21,596 dòng và 22 cột.


## 4. Tạo bảng tra cứu Zipcode (Support for App Demo)
Tính toán tọa độ trung tâm (`lat`, `long`) của từng mã bưu chính (`zipcode`). Bảng này sẽ được sử dụng trong Ứng dụng Demo để tự động điền tọa độ khi người dùng chọn khu vực.

In [ ]:
# Aggregation để lấy tọa độ trung bình
zip_lookup = df_final.groupBy("zipcode").agg(
    F.avg("lat").alias("center_lat"),
    F.avg("long").alias("center_long")
).toPandas()


## 6. Lưu trữ & Bàn giao
Lưu dữ liệu sạch dưới định dạng **Parquet** (định dạng tối ưu cho Big Data) để chuyển giao

In [ ]:
# --- 1. KẾT NỐI GOOGLE DRIVE ---
from google.colab import drive
drive.mount('/content/drive')


# --- 3. LƯU KẾT QUẢ VÀO DRIVE (Thay vì lưu tạm) ---
# Lưu vào đường dẫn '/content/drive/MyDrive/...'
output_path_1 = "/content/drive/MyDrive/dulieulon/BTL/processed_house_data.parquet"
output_path_2 = "/content/drive/MyDrive/dulieulon/BTL/zipcode_coords.csv"

df_final.write.mode("overwrite").parquet(output_path_1)
zip_lookup.to_csv(output_path_2, index=False)

print(f"✅ Đã lưu dữ liệu sạch vào Google Drive: {output_path_1}")
print(f"✅ Đã lưu tọa độ trung tâm vào Google Drive: {output_path_2}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Đã lưu dữ liệu sạch vào Google Drive: /content/drive/MyDrive/dulieulon/BTL/processed_house_data.parquet
✅ Đã lưu tọa độ trung tâm vào Google Drive: /content/drive/MyDrive/dulieulon/BTL/zipcode_coords.csv
